# Pivoting

From ChatGPT
Did not run because indmax has been deprecated in favor of argmax. I am pretty sure it is not going to work with argmax because it returns a CartesianIndex.  I had to cast CartesianIndex to a Tuple.

In [104]:
function lu_complete_pivoting!(A)
    n = size(A, 1)
    p = collect(1:n) # Permutation vector for rows
    q = collect(1:n) # Permutation vector for columns
    for k = 1:n-1
        # Find the index (i,j) of the pivot element
        # Edited to argmax since indmax is deprecated
        # Cast to Tuple because argmax returned a CartesianIndex structure
        # Original code was "i_max, j_max = argmax(abs.(A[k:n, k:n]))" 
        i_max, j_max = Tuple(argmax(abs.(A[k:n, k:n])))
        i_max += k - 1
        j_max += k - 1

        if i_max != k
            A[[k, i_max], :] = A[[i_max, k], :]
            p[[k, i_max]] = p[[i_max, k]]
        end

        if j_max != k
            A[:, [k, j_max]] = A[:, [j_max, k]]
            q[[k, j_max]] = q[[j_max, k]]
        end

        for i = k+1:n
            A[i, k] /= A[k, k]
            for j = k+1:n
                A[i, j] -= A[i, k] * A[k, j]
            end
        end
    end
    return A, p, q
end

lu_complete_pivoting! (generic function with 1 method)

Testing

In [127]:
using LinearAlgebra, SparseArrays
m=5
A0=rand(m,m)
b=rand(m)
A=copy(A0)
LU, p, q = lu_complete_pivoting!(A)
# reconstruct L and U
L=tril(LU,-1)+I; U=triu(LU)
println("||A0[p,q]-L*U|| = ", norm(A0[p,q]-L*U))
# Contruct P and Q 
Id=1.0*sparse(I, m, m)
P=copy(Id)
Q=copy(Id)
P=P[:,p]
Q=Q[:,q]
println("||P'*A0*Q-L*U|| = ", norm(P'*A0*Q-L*U))

||A0[p,q]-L*U|| = 1.3597399555105182e-16
||P'*A0*Q-L*U|| = 1.3597399555105182e-16


# ILU0

ChatGPT gave me an LU not an ILU0 that worked on and returned sparse arrays.  It 

In [130]:
using LinearAlgebra, SparseArrays
# modified type test
# originally "function Ailu0(A::SparseMatrixCSC{T, Int} where T)"
# Explicitly typed the spzeros to Float64 in decleration of L and U 
function Ailu0(A)
    n = size(A, 1)
    L = spzeros(Float64, n, n)
    U = spzeros(Float64, n, n)
    
    for k = 1:n
        L[k, k] = 1.0
        U[k, k] = A[k, k] - dot(L[k, 1:k-1], U[1:k-1, k])
        for i = k+1:n
            L[i, k] = (A[i, k] - dot(L[i, 1:k-1], U[1:k-1, k])) / U[k, k]
        end
        for j = k+1:n
            U[k, j] = A[k, j] - dot(L[k, 1:k-1], U[1:k-1, j])
        end
    end
    
    return L, U
end

Ailu0 (generic function with 1 method)

The fill in is clear on our test matrix. 

In [138]:
using MatrixMarket
A=mmread("C:\\Users\\AllanStruthers\\Desktop\\GitHub\\23-24-Classes\\Spring 2024 Courses\\Lin Alg\\Projects\\ILU0\\ILU0Test.mtx")
(L,U) = Ailu0(A)
display(A)
display(L)
display(U)
norm(A-L*U)

59×59 SparseMatrixCSC{Float64, Int64} with 313 stored entries:
⎡⠑⢄⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠑⢄⠈⠢⡀⠐⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠑⣤⠬⠆⠀⠑⣀⣀⡀⠀⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⣿⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⣿⣿⡏⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣍⠁⠀⠈⠐⢄⠀⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⢀⣀⣀⠀⠈⠓⢄⠀⠀⠀⠑⠀⠈⢢⠤⠤⠄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⠀⠀⠀⠀⠀⠀⠑⢄⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⢡⣤⣤⠰⠒⠒⡄⠀⠀⠀⎥
⎢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⣿⣿⠀⠑⠄⡇⠀⠀⠀⎥
⎢⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⢄⠀⠀⡇⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠤⠤⠘⢍⠉⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠁⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⡀⠀⠀⠀⠑⠀⠀⠀⠀⠀⠀⠀⠀⠱⣄⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⠄⎦

59×59 SparseMatrixCSC{Float64, Int64} with 606 stored entries:
⎡⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠑⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⢀⣀⣀⣀⣈⣓⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡀⠀⢀⠀⠀⣀⣀⣀⠀⠀⣀⣀⡀⠀⢀⣀⣀⣀⣀⣀⣱⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠈⠢⡀⠑⢄⠈⠻⡇⠑⢄⣿⣿⡏⠢⣼⣿⣿⣿⣿⣿⣿⣿⣷⣄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠤⠤⠼⢿⣿⣿⣿⣿⣿⣿⣿⣷⣄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⡀⠠⠤⠤⠽⠿⠿⢿⣿⣿⣿⣿⣿⡷⣄⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠇⠈⠳⠄⎦

59×59 SparseMatrixCSC{Float64, Int64} with 428 stored entries:
⎡⠑⢄⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠑⢄⠈⠢⡀⠐⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠑⢤⣬⡆⠀⠑⣀⣀⡀⠀⠀⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠙⢇⠀⠀⣿⣿⡇⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⣿⣿⡏⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⡇⠀⢸⢰⣄⠀⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⣘⣘⣛⣓⣛⣛⣢⣤⣤⡄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣿⣿⣿⣿⣿⡇⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣿⣿⣿⡇⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢍⢩⣭⣭⠱⠒⠒⡄⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣀⣱⣄⡇⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⡇⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⡇⠀⠀⡇⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⡇⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⠇⎦

5.752123344641334e-17

To make this code (as is an ILU0) requires a lot of if conditions.  This is fine if you have it done this way.  Just be prepared for questions about efficiency.  